# Gridded EPA Methane Inventory
## Category: 1B2b Natural Gas Distribution

***
#### Authors: 
Joannes D. Maasakkers, Candice F. Z. Chen, Erin E. McDuffie
#### Date Last Updated: 
See Step 0
#### Notebook Purpose: 
This Notebook calculates and reports annual gridded (0.1⁰x0.1⁰) methane emission fluxes (molec./cm2/s) from natural gas systems distribution segment in the CONUS region between 2012-2018. 
#### Summary & Notes:
EPA GHGI gas distribution emissions are read in from the GHGI Natural Gas Systems workbook at the national level. Emissions are first allocated to the state level as a function of emission group. The activity/proxy data used to allocate emissions from each group include Pipeline and Hazardous materials Safety Administration (PHMSA) distribution main pipeline mileage (by material) and service counts, as well as metering station counts from GHGRP and EIA customer meters. State-level emissions are then spatially distributed onto a 0.01x0.01 degree grid based on population density. Emissions are regridded to a 0.1x0.1 degree grid and converted to emission flux. Annual emission fluxes (molec./cm2/s) are written to final netCDFs in the ‘/code/Final_Gridded_Data/’ folder. 
***

######

-------
## Step 0. Set-Up Notebook Modules, Functions, and Local Parameters and Constants
_____

In [ ]:
#Confirm working directory
import os
import time
modtime = os.path.getmtime('./1B2b_Distribution.ipynb')
modificationTime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(modtime))
print("This file was last modified on: ", modificationTime)
print('')
print("The directory we are working in is {}" .format(os.getcwd()))

In [ ]:
## Include plots within notebook
%matplotlib inline

In [ ]:
# Import base modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import datetime
from copy import copy

# Import additional modules
from mpl_toolkits.basemap import Basemap

# Load netCDF (for manipulating netCDF file types)
from netCDF4 import Dataset

# Set up ticker
import matplotlib.ticker as ticker

#add path for the global function module (file)
import sys
module_path = os.path.abspath(os.path.join('../Global_Functions/'))
#print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

# Load functions
import data_load_functions as data_load_fn
import data_functions as data_fn
import data_IO_functions as data_IO_fn
import data_plot_functions as data_plot_fn

In [ ]:
#INPUT Files
# Assign global file names
global_filenames = data_load_fn.load_global_file_names()
State_ANSI_inputfile = global_filenames[0]
#County_ANSI_inputfile = global_filenames[1]
pop_map_inputfile = global_filenames[2]
Grid_area01_inputfile = global_filenames[3]
Grid_area001_inputfile = global_filenames[4]
Grid_state001_ansi_inputfile = global_filenames[5]
#Grid_county001_ansi_inputfile = global_filenames[6]
globalinputlocation = global_filenames[0][0:20]
print(globalinputlocation)

# Specify names of inputs files used in this notebook
EPA_NG_inputfile = globalinputlocation+'GHGI/Ch3_Energy/NaturalGasSystems_1990-2018_GHGI_2020-04-11.xlsx'

#proxy mapping file
NG_Mapping_inputfile = './InputData/NaturalGas_Distribution_ProxyMapping.xlsx'

#Activity Data
PHMSA_inputfile = './InputData/annual_gas_distr.csv'
EIA_Residential_CC_inputfile = './InputData/EIA_CustomerCounts/NG_CONS_NUM_A_EPG0_VN3_COUNT_A.xls'
EIA_Commercial_CC_inputfile = './InputData/EIA_CustomerCounts/NG_CONS_NUM_A_EPG0_VN5_COUNT_A.xls'
EIA_Industrial_CC_inputfile = './InputData/EIA_CustomerCounts/NG_CONS_NUM_A_EPG0_VN7_COUNT_A.xls'

phmsa_gas_distr_2012_inputfile = './InputData/PHMSA_annual_gas_distribution_2012.csv'

#GHGRP Data (reporting format changed in 2015)
GHGRP_facility_inputfile = './InputData/ghgrp_facility_info.CSV'
ghgrp_distr_2014_inputfile = './InputData/metering-2014.CSV' # pipeline mileage and above and below ground metering and serivce stations
ghgrp_distr_meteringbelow_2018_inputfile = './InputData/metering-2018.CSV' #pipeline mileage and below-grade station equipent counts
ghgrp_distr_meteringabove_2018_inputfile = './InputData/metering_above-2018.CSV' #above-grade station counts

#OUTPUT FILES
gridded_outputfile = '../Final_Gridded_Data/EPA_v2_1B2b_Natural_Gas_Distribution.nc'
netCDF_description = 'Gridded EPA Inventory - Natural Gas Systems Emissions - IPCC Source Category 1B2b - Distribution'
title_str = "EPA methane emissions from gas distribution"
title_diff_str = "Emissions from gas distribution difference: 2018-2012"

#output gridded proxy data
grid_emi_outputfile = '../Final_Gridded_Data/Extension/v2_input_data/NG_Distribution_Grid_Emi.nc'

In [ ]:
# Define local variables
start_year = 2012  #First year in emission timeseries
end_year = 2018    #Last year in emission timeseries
year_range = [*range(start_year, end_year+1,1)] #List of emission years
year_range_str=[str(i) for i in year_range]
num_years = len(year_range)

# Define constants
Avogadro   = 6.02214129 * 10**(23)  #molecules/mol
Molarch4   = 16.04                  #g/mol
Res01      = 0.1                    # degrees

# Continental US Lat/Lon Limits (for netCDF files)
Lon_left = -130       #deg
Lon_right = -60       #deg
Lat_low  = 20         #deg
Lat_up  = 55          #deg
loc_dimensions = [Lat_low, Lat_up, Lon_left, Lon_right]

ilat_start = int((90+Lat_low)/Res01) #1100:1450 (continental US range)
ilat_end = int((90+Lat_up)/Res01)
ilon_start = abs(int((-180-Lon_left)/Res01)) #500:1200 (continental US range)
ilon_end = abs(int((-180-Lon_right)/Res01))

# Number of days in each month
month_day_leap  = [  31,  29,  31,  30,  31,  30,  31,  31,  30,  31,  30,  31]
month_day_nonleap = [  31,  28,  31,  30,  31,  30,  31,  31,  30,  31,  30,  31]
month_tag = ['01','02','03','04','05','06','07','08','09','10','11','12']
month_dict = {'January':1, 'February':2,'March':3,'April':4,'May':5,'June':6, 'July':7,'August':8,'September':9,'October':10,\
             'November':11,'December':12}

# Month arrays
month_range_str = ['January','February','March','April','May','June','July','August','September','October','November','December']
num_months = len(month_range_str)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;
//prevent auto-scrolling

In [ ]:
# Track run time
ct = datetime.datetime.now() 
it = ct.timestamp() 
print("current time:", ct) 

____
## Step 1. Load in State ANSI data and Area
_____

In [ ]:
# State-level ANSI Data
#Read the state ANSI file array
State_ANSI, name_dict = data_load_fn.load_state_ansi(State_ANSI_inputfile)[0:2]
#QA: number of states
print('Read input file: '+ f"{State_ANSI_inputfile}")
print('Total "States" found: ' + '%.0f' % len(State_ANSI))
print(' ')

# 0.01 x0.01 degree Data
# State ANSI IDs and grid cell area (m2) maps
state_ANSI_map = data_load_fn.load_state_ansi_map(Grid_state001_ansi_inputfile)
area_map, lat001, lon001 = data_load_fn.load_area_map_001(Grid_area001_inputfile)

# 0.1 x0.1 degree data
# grid cell area and state ANSI maps
Lat01, Lon01 = data_load_fn.load_area_map_01(Grid_area01_inputfile)[1:3]
#Select relevant Continental 0.1 x0.1 domain
Lat_01 = Lat01[ilat_start:ilat_end]
Lon_01 = Lon01[ilon_start:ilon_end]
area_matrix_01 = data_fn.regrid001_to_01(area_map, Lat_01, Lon_01)
area_matrix_01 *= 10000  #convert from m2 to cm2
state_ANSI_map_01 = data_fn.regrid001_to_01(state_ANSI_map, Lat_01, Lon_01)
del lat001, lon001

# Print time
ct = datetime.datetime.now() 
print("current time:", ct) 

-------------
## Step 2: Read in and Format Proxy Data
-------------

#### Step 2.1 Read In Proxy Mapping File & Make Proxy Arrays

#### Step 2.1.1 Format Proxy Group Arrays

In [ ]:
#load GHGI Mapping Groups
names = pd.read_excel(NG_Mapping_inputfile, sheet_name = "GHGI Map - Dist", usecols = "A:B",skiprows = 1, header = 0)
colnames = names.columns.values
ghgi_dist_map = pd.read_excel(NG_Mapping_inputfile, sheet_name = "GHGI Map - Dist", usecols = "A:B", skiprows = 2, names = colnames)
#drop rows with no data, remove the parentheses and ""
ghgi_dist_map = ghgi_dist_map[ghgi_dist_map['GHGI_Emi_Group'] != 'na']
ghgi_dist_map = ghgi_dist_map[ghgi_dist_map['GHGI_Emi_Group'].notna()]
ghgi_dist_map = ghgi_dist_map[ghgi_dist_map['GHGI_Emi_Group'] != '-']
ghgi_dist_map['GHGI_Source']= ghgi_dist_map['GHGI_Source'].str.replace(r"\(","")
ghgi_dist_map['GHGI_Source']= ghgi_dist_map['GHGI_Source'].str.replace(r"\)","")
ghgi_dist_map.reset_index(inplace=True, drop=True)
display(ghgi_dist_map)

#load emission group - proxy map
names = pd.read_excel(NG_Mapping_inputfile, sheet_name = "Proxy Map - Dist", usecols = "A:D",skiprows = 1, header = 0)
colnames = names.columns.values
proxy_dist_map = pd.read_excel(NG_Mapping_inputfile, sheet_name = "Proxy Map - Dist", usecols = "A:D", skiprows = 1, names = colnames)
display((proxy_dist_map))

#create empty proxy and emission group arrays (add months for proxy variables that have monthly data)
for igroup in np.arange(0,len(proxy_dist_map)):
        vars()[proxy_dist_map.loc[igroup,'Proxy_Group']] = np.zeros((len(State_ANSI),num_years))
        vars()[proxy_dist_map.loc[igroup,'Proxy_Group']+'_nongrid'] = np.zeros([num_years])
        vars()[proxy_dist_map.loc[igroup,'GHGI_Emi_Group']] = np.zeros([num_years])
        if proxy_dist_map.loc[igroup,'State_Proxy_Group'] != '-':
            vars()[proxy_dist_map.loc[igroup,'State_Proxy_Group']] = np.zeros([len(State_ANSI),num_years])
            
emi_group_names = np.unique(ghgi_dist_map['GHGI_Emi_Group'])
print('QA/QC: Is the number of emission groups the same for the proxy and emissions tabs?')
if (len(emi_group_names) == len(np.unique(proxy_dist_map['GHGI_Emi_Group']))):
    print('PASS')
else:
    print('FAIL')

#### Step 2.2 - Read in 0.01x0.01 State population data and format State_ANSI

In [ ]:
# Read population maps

#Read population map
pop_den_map = data_load_fn.load_pop_den_map(pop_map_inputfile)

# Calculate the population totals per state (will only include continental US)
# Population totals for each state = the sum of the grid cell product of the population density in each state * the area of each state

State_ANSI['Population'] = 0.0

for istate in np.arange(len(State_ANSI)):
    State_ANSI.loc[istate, 'Population'] = np.sum(pop_den_map[state_ANSI_map == State_ANSI['ansi'][istate]]*area_map[state_ANSI_map == State_ANSI['ansi'][istate]])

# Check that state-level population array is correct
print('QA/QC: Check State-level populations are correct')
if abs((np.sum(pop_den_map[state_ANSI_map != 0]*area_map[state_ANSI_map != 0]) - np.sum(State_ANSI['Population']))/float(np.sum(pop_den_map[state_ANSI_map != 0]*area_map[state_ANSI_map != 0]))) <1e-2:
    print('PASS')
else:
    print('FAIL')

#### Step 2.3 Read In PHMSA Pipeline Mileage and Services Data 

In [ ]:
# Read company level PHMSA data
PHMSA_facilities = pd.read_csv(PHMSA_inputfile, sep=',',low_memory=False)

# make new array of PHMSA pipeline miles data
PHMSA_facilities['REPORT_YEAR'] = PHMSA_facilities['REPORT_YEAR'].astype(int)
PHMSA_facilities['MMILES_CI'] = PHMSA_facilities['MMILES_CI'].astype(float)
PHMSA_facilities['MMILES_STEEL_UNP_BARE'] = PHMSA_facilities['MMILES_STEEL_UNP_BARE'].astype(float)
PHMSA_facilities['MMILES_STEEL_UNP_COATED'] = PHMSA_facilities['MMILES_STEEL_UNP_COATED'].astype(float)
PHMSA_facilities['MMILES_STEEL_CP_BARE'] = PHMSA_facilities['MMILES_STEEL_CP_BARE'].astype(float)
PHMSA_facilities['MMILES_STEEL_CP_COATED'] = PHMSA_facilities['MMILES_STEEL_CP_COATED'].astype(float)
PHMSA_facilities['MMILES_PLASTIC'] = PHMSA_facilities['MMILES_PLASTIC'].astype(float)
PHMSA_facilities['NUM_SRVS_STEEL_UNP_BARE'] = PHMSA_facilities['NUM_SRVS_STEEL_UNP_BARE'].astype(float)
PHMSA_facilities['NUM_SRVS_STEEL_UNP_COATED'] = PHMSA_facilities['NUM_SRVS_STEEL_UNP_COATED'].astype(float)
PHMSA_facilities['NUM_SRVS_STEEL_CP_BARE'] = PHMSA_facilities['NUM_SRVS_STEEL_CP_BARE'].astype(float)
PHMSA_facilities['NUM_SRVS_STEEL_CP_COATED'] = PHMSA_facilities['NUM_SRVS_STEEL_CP_COATED'].astype(float)
PHMSA_facilities['NUM_SRVS_PLASTIC'] = PHMSA_facilities['NUM_SRVS_PLASTIC'].astype(float)
PHMSA_facilities['NUM_SRVS_CU'] = PHMSA_facilities['NUM_SRVS_CU'].astype(float)

# Format Pipeline Mileage Data
# Create arrays to hold main and service mileages with dimensions (state X year)
main_castiron = np.zeros((len(State_ANSI),num_years))
main_usteel = np.zeros((len(State_ANSI),num_years))
main_psteel = np.zeros((len(State_ANSI),num_years))
main_plastic = np.zeros((len(State_ANSI),num_years))

serv_usteel = np.zeros((len(State_ANSI),num_years))
serv_psteel = np.zeros((len(State_ANSI),num_years))
serv_plastic = np.zeros((len(State_ANSI),num_years))
serv_copper = np.zeros((len(State_ANSI),num_years))

# for each year, sum all the mileage data across all piepline and service types
# (e.g., cast iron, unprotected steel, etc.), then group the facility level data by state
for iyear in np.arange(num_years):

    mcastiron_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['MMILES_CI'].values
    musteel_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['MMILES_STEEL_UNP_BARE'].values + PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['MMILES_STEEL_UNP_COATED'].values
    mpsteel_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['MMILES_STEEL_CP_BARE'].values + PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['MMILES_STEEL_CP_COATED'].values
    mplastic_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['MMILES_PLASTIC'].values
    
    # Print data
    print('Total mileage '+str(iyear+start_year)+':') 
    print('Cast Iron: ', np.sum(mcastiron_temp))
    print('Unprotected steel: ', np.sum(musteel_temp))
    print('Protected steel: ', np.sum(mpsteel_temp))
    print('Plastic: ', np.sum(mplastic_temp))         
    print(' ')
    
    susteel_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['NUM_SRVS_STEEL_UNP_BARE'].values + PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['NUM_SRVS_STEEL_UNP_COATED'].values
    spsteel_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['NUM_SRVS_STEEL_CP_BARE'].values + PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['NUM_SRVS_STEEL_CP_COATED'].values
    splastic_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['NUM_SRVS_PLASTIC'].values
    scopper_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]['NUM_SRVS_CU'].values
          
    # Print data
    print('Services '+str(iyear+start_year)+':')
    print('Unprotected steel: ', np.sum(susteel_temp))
    print('Protected steel: ', np.sum(spsteel_temp))
    print('Plastic: ', np.sum(splastic_temp))
    print('Copper: ', np.sum(scopper_temp))
    print(' ')
        
    # get state for each PHMSA facilities
    PHMSA_facilities_temp = PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year]
    PHMSA_facilities_temp.reset_index(drop=True,inplace=True)

    # Calculate the mileage and services for each state
    for ifacility in np.arange(len(PHMSA_facilities[PHMSA_facilities['REPORT_YEAR'] == iyear+start_year])):
        match_state = np.where(State_ANSI['abbr'] == PHMSA_facilities_temp['STOP'][ifacility])[0][0]
        #print(State_ANSI['abbr'][match_state]+', '+str(match_state))
        
        #Add mileage
        main_castiron[match_state][iyear] += mcastiron_temp[ifacility]
        main_usteel[match_state][iyear] += musteel_temp[ifacility]
        main_psteel[match_state][iyear] += mpsteel_temp[ifacility]
        main_plastic[match_state][iyear] += mplastic_temp[ifacility]

        #Add services
        serv_usteel[match_state][iyear] += susteel_temp[ifacility]
        serv_psteel[match_state][iyear] += spsteel_temp[ifacility]
        serv_plastic[match_state][iyear] += splastic_temp[ifacility]
        serv_copper[match_state][iyear] += scopper_temp[ifacility]
        
# Calculate PHMSA mains total 
PHMSA_mains_total = main_castiron + main_usteel + main_psteel + main_plastic

#### Step 2.4 Read In State-Level EIA Customer Counts Data

In [ ]:
# Read company level EIA data, but only load the totals for all companies across each state
EIA_ResCounts = pd.read_excel(EIA_Residential_CC_inputfile, sheet_name = 'Data 1', skiprows=2)
EIA_ComCounts = pd.read_excel(EIA_Commercial_CC_inputfile, sheet_name = 'Data 1', skiprows=2)
EIA_IndCounts = pd.read_excel(EIA_Industrial_CC_inputfile, sheet_name = 'Data 1', skiprows=2)

#convert the time stamp to year
EIA_ResCounts['Date'] = EIA_ResCounts['Date'].astype(str)
EIA_ComCounts['Date'] = EIA_ComCounts['Date'].astype(str)
EIA_IndCounts['Date'] = EIA_IndCounts['Date'].astype(str)
EIA_ResCounts['Date'] = [EIA_ResCounts['Date'][i][0:4] for i in np.arange(len(EIA_ResCounts))]   #extract the year
EIA_ComCounts['Date'] = [EIA_ComCounts['Date'][i][0:4] for i in np.arange(len(EIA_ComCounts))]   #extract the year
EIA_IndCounts['Date'] = [EIA_IndCounts['Date'][i][0:4] for i in np.arange(len(EIA_IndCounts))]   #extract the year
#transpose and reset the column and row indexes
EIA_ResCounts = EIA_ResCounts.T
EIA_ComCounts = EIA_ComCounts.T
EIA_IndCounts = EIA_IndCounts.T
EIA_ResCounts.columns = EIA_ResCounts.iloc[0]
EIA_ComCounts.columns = EIA_ComCounts.iloc[0]
EIA_IndCounts.columns = EIA_IndCounts.iloc[0]
EIA_ResCounts = EIA_ResCounts.drop(EIA_ResCounts.index[[0,1]])
EIA_ComCounts = EIA_ComCounts.drop(EIA_ComCounts.index[[0,1]])
EIA_IndCounts = EIA_IndCounts.drop(EIA_IndCounts.index[[0,1]])
#extract the state names and format as abbreviations
Names_res = EIA_ResCounts.index.values.tolist()
Names_com = EIA_ComCounts.index.values.tolist()
Names_ind = EIA_IndCounts.index.values.tolist()
EIA_ResCounts.reset_index(drop=True,inplace=True)
EIA_ComCounts.reset_index(drop=True,inplace=True)
EIA_IndCounts.reset_index(drop=True,inplace=True)
EIA_ResCounts['State'] = Names_res
EIA_ComCounts['State'] = Names_com
EIA_IndCounts['State'] = Names_ind
    
for istate in np.arange(0,len(State_ANSI)-6): #### -6
    #print(State_ANSI['name'][istate])
    match_state = np.where(EIA_ResCounts['State'].str.contains(State_ANSI['name'][istate]))[0][0]
    EIA_ResCounts['State'][match_state] = State_ANSI['abbr'][istate]
    #print(match_state)
    match_state = np.where(EIA_ComCounts['State'].str.contains(State_ANSI['name'][istate]))[0][0]
    EIA_ComCounts['State'][match_state] = State_ANSI['abbr'][istate]
    #print(match_state)
    if State_ANSI['name'][istate]=='District of Columbia':
        #print(len(EIA_IndCounts.loc[0]))
        EIA_IndCounts.loc[-1] = np.zeros([len(EIA_IndCounts.loc[0])])
        EIA_IndCounts.index = EIA_IndCounts.index+1
        EIA_IndCounts = EIA_IndCounts.sort_index()
        EIA_IndCounts['State'][0] = State_ANSI['abbr'][istate]
        #print(EIA_IndCounts.head(20))
    else:
        match_state = np.where(EIA_IndCounts['State'].str.contains(State_ANSI['name'][istate]))[0][0]
        EIA_IndCounts['State'][match_state] = State_ANSI['abbr'][istate]

# Initialize state array of leak losses (State X year) 
res_counts = np.zeros((len(State_ANSI),num_years))
indcom_counts = np.zeros((len(State_ANSI),num_years))
#ind_counts = np.zeros((len(State_ANSI),num_years))

# To this array, add ANSI value for each state, then for each year,
# make a new leakvolume array that records the yearly state-level leak loss volume data
# Data only extend back to 1997, so use 1997 values for years 1990-1996

for ifacility in np.arange(0,len(EIA_ResCounts)):
    match_state1 = np.where(State_ANSI['abbr'] == EIA_ResCounts['State'][ifacility])[0][0]
    match_state2 = np.where(State_ANSI['abbr'] == EIA_ComCounts['State'][ifacility])[0][0]
    match_state3 = np.where(State_ANSI['abbr'] == EIA_IndCounts['State'][ifacility])[0][0]

    for iyear in np.arange(num_years):
        res_counts[match_state1][iyear] += EIA_ResCounts[str(iyear+start_year)][ifacility]
        indcom_counts[match_state2][iyear] += EIA_ComCounts[str(iyear+start_year)][ifacility]
        indcom_counts[match_state3][iyear] += EIA_IndCounts[str(iyear+start_year)][ifacility]

# Fill in data gaps (interpolate to fill zeros between years and extend most historical value back to 1990)
res_counts = np.nan_to_num(res_counts)
indcom_counts = np.nan_to_num(indcom_counts)


for iyear in np.arange(0,num_years):
    print(year_range_str[iyear]+' Total Counts: ')
    print(' Residential Customers: '+str(res_counts.sum(axis=0)[iyear]))
    print(' Commercial Customers: '+str(indcom_counts.sum(axis=0)[iyear]))

#### 2.5 Read in and Format GHGRP Metering Station Counts and Pipeline Mileage Data

###### 2.5.1 Read in and Format GHGRP Facility Level Data
Note: GHGRP reporting format changed in 2015. Current code uses data from the most recent year (2018) and the year before the reporting change (2014). 

In [ ]:
# Read in and Format GHGRP facility-level information by State

#a) Read in the GHGRP data
facility_info = pd.read_csv(GHGRP_facility_inputfile)
facility_2014 = pd.read_csv(ghgrp_distr_2014_inputfile)
facility_2018 = pd.read_csv(ghgrp_distr_meteringbelow_2018_inputfile) #pipeline mileage and below-grade stations
facility_above_2018 = pd.read_csv(ghgrp_distr_meteringabove_2018_inputfile) #above-grade stations


#b) Add state column to metering stations
# record the state abbreviation for each facility ID (for all three files)
facility_2014_state = []
facility_2018_state = []
facility_above_2018_state = []

# for each entry in the data file (each facility each year), match the facility ID in the 2014 data to the ID in the
# GHGRP facility info file, then append the corresponding state abbreviation to the facility_[year]_state array
for index in np.arange(len(facility_2014)):
    ilocation = np.where(facility_info['V_GHG_EMITTER_FACILITIES.FACILITY_ID'] == facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID'][index])[0][0]
    facility_2014_state.append(facility_info['V_GHG_EMITTER_FACILITIES.STATE'][ilocation])

for index in np.arange(len(facility_2018)):
    ilocation = np.where(facility_info['V_GHG_EMITTER_FACILITIES.FACILITY_ID'] == facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID'][index])[0][0]
    facility_2018_state.append(facility_info['V_GHG_EMITTER_FACILITIES.STATE'][ilocation])
    
for index in np.arange(len(facility_above_2018)):
    ilocation = np.where(facility_info['V_GHG_EMITTER_FACILITIES.FACILITY_ID'] == facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID'][index])[0][0]
    facility_above_2018_state.append(facility_info['V_GHG_EMITTER_FACILITIES.STATE'][ilocation])
    
# add the the state abbreviations to the main arrays
facility_2014['state'] = facility_2014_state
facility_2018['state'] = facility_2018_state
facility_above_2018['state'] = facility_above_2018_state


#c) Format GHGRP data
# Replace nan counts with zero
facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'] = facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'].fillna(0)
facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'] = facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'].fillna(0)
facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'] = facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'].fillna(0)

facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_CAST_IRON_DIST_MAINS'] = facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_CAST_IRON_DIST_MAINS'].fillna(0)
facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PLSTIC_DIST_MAINS'] = facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PLSTIC_DIST_MAINS'].fillna(0)
facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PROT_STEEL_DIST_MAINS'] = facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PROT_STEEL_DIST_MAINS'].fillna(0)
facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_UNPR_STEEL_DIST_MAINS'] = facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_UNPR_STEEL_DIST_MAINS'].fillna(0)

# Filter out null rows
facility_2014.dropna(subset=['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR'],inplace=True)
facility_2014.reset_index(drop=True,inplace=True)
facility_2018.dropna(subset=['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'],inplace=True)
facility_2018.reset_index(drop=True,inplace=True)
facility_above_2018.dropna(subset=['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR'],inplace=True) 
facility_above_2018.reset_index(drop=True,inplace=True)


#d) Filter out GHGRP IDs that don't report in all 7 years (for consistency)
# Check within 2011-2014
for index in np.arange(len(facility_2014)):
    ID = facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID'][index]
    if len(facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID'] == ID]) < 4:
        facility_2014.drop([index],axis=0,inplace=True)
facility_2014.reset_index(drop=True,inplace=True)

# Check within 2015-2018 (above-grade)
for index in np.arange(len(facility_above_2018)):
    ID = facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID'][index]
    if len(facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID'] == ID]) < 4:
        facility_above_2018.drop([index],axis=0,inplace=True)
facility_above_2018.reset_index(drop=True,inplace=True)

# Check within 2015-2018 (below-grade)
facility_filtered = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'] == 'Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig']
for index in np.arange(len(facility_2018)):
    ID = facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID'][index]
    if len(facility_filtered[facility_filtered['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID'] == ID]) < 4:
        facility_2018.drop([index],axis=0,inplace=True)
facility_2018.reset_index(drop=True,inplace=True)

# Checking between datasets (i.e, drop facilities that are not reported both before and after 2015)
# Direction: Old to new
for index in np.arange(len(facility_2014)):
    ID = facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID'][index]
    if len(facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==ID])==0:
        facility_2014.drop([index],axis=0,inplace=True)
facility_2014.reset_index(drop=True,inplace=True)

# Direction: New to old, above ground
for index in np.arange(len(facility_above_2018)):
    ID = facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID'][index]
    if len(facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==ID])==0:
        facility_above_2018.drop([index],axis=0,inplace=True)
facility_above_2018.reset_index(drop=True,inplace=True)        

# Direction: New to old, below 
for index in np.arange(len(facility_2018)):
    ID = facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID'][index]
    if len(facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==ID])==0:
        facility_2018.drop([index],axis=0,inplace=True)
facility_2018.reset_index(drop=True,inplace=True)

# Print time
ct = datetime.datetime.now() 
print("current time:", ct) 

##### 2.5.2 Fixing GHGRP Data Inconsistencies in Stations Reporting
There are a few reporting inconsistencies in the GHGRP data. 
Below are 7 known problems, the causes, EPA's proposed solutions, and their implementations.

In [ ]:
# Problem 1: 
# AL has anomalous drop in stations in 2013 and 2014. 
# Cause: 
# facility 1005967 didn't report below-grade metering stations in 2013-2014 but picked back up 2015-2018.
# EPA Solution:
# Interpolation is likely the best approach. The facility appears to have omitted the count of below 
# grade stations (both T-D and M/R all together from their Reporting Year (RY) 2013 and RY 2014 annual reports). 
# We note the report includes emissions from these sources, so only the station count is missing.

# Get linear fit using the start year (2012), and 2015 to end year (2018) points, then interpolate 2013-2014
# Create array to hold annual stations count

stations = np.zeros(num_years)

# 2012
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2012]
facility_temp = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1005967]
stations[0] = facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_METERING_STATIONS'].iloc[0] + \
    facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_TRANSFER_STATIONS'].iloc[0] + \
    facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS'].iloc[0] + \
    facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_TRANSFER_STATIONS'].iloc[0]

# Below-grade 2015-2018 (end year)
for iyear in np.arange((2015-start_year),num_years):
    facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'] == iyear+start_year]
    facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID']==1005967]
    facility_temp.reset_index(drop=True,inplace=True)
    
    for index in np.arange(len(facility_temp)):
        if facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig':   
            stations[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure 100 to 300 psig':   
            stations[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index] 
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure < 100 psig':   
            stations[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure > 300 psig':   
            stations[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure 100 to 300 psig':   
            stations[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure < 100 psig':   
            stations[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]

# Above-grade 2015-end year(2018)
for iyear in np.arange((2015-start_year),num_years):
    facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR'] == iyear+start_year]
    facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1005967]
    facility_temp.reset_index(drop=True,inplace=True)
    
    for index in np.arange(len(facility_temp)):
        stations[iyear] += facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'][index]
        stations[iyear] += facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'][index]

# Linear fit
x = [start_year]
x.extend(list(range(2015,end_year+1))) #x=years
match_years = [year_range.index(i) for i in x] # find which years these include (out of the entire range)
y = [stations[i] for i in match_years] # find the station data for those given years
coef = np.polyfit(x, y, 1)

# Calculate missing years: 2013, 2014
stations[1] = round(2013*coef[0] + coef[1]) #2013
stations[2] = round(2014*coef[0] + coef[1]) #2014

# Subtract above-grade stations in 2013/2014 to get only below-grade
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2013]
facility_temp = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1005967]
above_13 = facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_METERING_STATIONS'].iloc[0] + \
    facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_TRANSFER_STATIONS'].iloc[0]
below_13 = stations[1] - above_13
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2014]
facility_temp = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1005967]
above_14 = facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_METERING_STATIONS'].iloc[0] + \
    facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_TRANSFER_STATIONS'].iloc[0]
below_14 = stations[2] - above_14

# Find indices of 2013/2014 rows for company 1005967
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1005967]
index_2013 = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2013].index.values.astype(int)[0]
index_2014 = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2014].index.values.astype(int)[0]

# Put new 2013/2014 below-grade values into facility_14
facility_2014.at[index_2013,'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS']= below_13
facility_2014.at[index_2014,'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS']= below_14

In [ ]:
# Problem 2: 
# AK had low station counts in 2015-2018; 
# Cause: 
# facility 1006604 stopped reporting metering stations 2015-2018 (but still reported transfer stations).
# EPA Solution:
# Estimate metering station counts after 2015 using 2012 (start year)-2014 data.

# Get linear fit using 2012 (start year)-2014 to extrapolate 2015-2018 (end year) stations.

# Create array to hold annual stations count
stations = np.zeros(num_years)

# 2012-2014 stations
for iyear in np.arange(2015-start_year):
    facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==iyear+start_year]
    facility_temp = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1006604]
    stations[iyear] = facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_METERING_STATIONS'].iloc[0] + \
        facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_TRANSFER_STATIONS'].iloc[0] + \
        facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS'].iloc[0] + \
        facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_TRANSFER_STATIONS'].iloc[0]

# Linear fit
x = list(range(start_year, 2015)) #x = years (start year to 2014)
match_years = [year_range.index(i) for i in x] # find which years these include (out of the entire range)
y = [stations[i] for i in match_years] # find the station data for those given years
coef = np.polyfit(x, y, 1)

# Calculate station counts for 2015-2018 using previous relationship
index_2015 = year_range.index(2015)
for iyear in np.arange(end_year-2014):
    stations[index_2015+iyear] = round((2015+iyear)*coef[0] + coef[1])

# Count stations company reported 2015-2018
stations_before = np.zeros(num_years)
for iyear in np.arange(2015-start_year,num_years): # Below-grade 2015-2018
    facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'] == iyear+start_year]
    facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID']==1006604]
    facility_temp.reset_index(drop=True,inplace=True)
    for index in np.arange(len(facility_temp)):
        if facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig':   
            stations_before[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure 100 to 300 psig':   
            stations_before[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index] 
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure < 100 psig':   
            stations_before[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure > 300 psig':   
            stations_before[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure 100 to 300 psig':   
            stations_before[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure < 100 psig':   
            stations_before[iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
for iyear in np.arange(2015-start_year,num_years): # Above-grade 2015-2018
    facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR'] == iyear+start_year]
    facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1006604]
    facility_temp.reset_index(drop=True,inplace=True)
    
    for index in np.arange(len(facility_temp)):
        stations_before[iyear] += facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'][index]
        stations_before[iyear] += facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'][index]

# Subtract stations_before from stations to get how many stations need to be added in 2015-2018
stations_add = stations - stations_before

# Find 2015-2018 indices for company 1006604 to add additional stations (adding to above-grade metering stations)
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1006604]

for iyear in np.arange(end_year-2014):
    index = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR']==2015+iyear].index.values.astype(int)[0]
    facility_above_2018.at[index,'EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS']= stations_add[index_2015+iyear]

In [ ]:
# Problem 3:
# MS has implausible rise in reported stations in 2018 without additional mileage in 2018;
# Cause: 
# facility 1001688 reported 1552 more stations from 2017-2018.
# EPA Solution:
# Use 2017 value for 2018

# This problem is in facility_above_2018. We will change metering and transfer stations in 2018 to values in 2017

# Retrieve 2017 values
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1001688]
facility_17 = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR']==2017]
metering_17 = facility_17['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS']
transfer_17 = facility_17['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS']

# Retrieve 2018 index
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1001688]
facility_18 = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR']==2018]
index_18 = facility_18.index.values.astype(int)[0]

# Replace 2018 values with 2017 values
facility_above_2018.at[index_18,'EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'] = metering_17
facility_above_2018.at[index_18,'EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'] = transfer_17

In [ ]:
# Problem 4:
# NJ below-grade metering stations are too low in 2014
# Cuase: 
# NJ facility 1002812 didn’t report below-grade metering stations in 2014.
# EPA Solution:
# Interpolation between 2013 and 2015.

# 2014 value is average of 2013 and 2015 below-grade stations.

# 2015 below-grade metering value
facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'] == 2015]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID']==1002812]
facility_temp.reset_index(drop=True,inplace=True)
metering_15 = 0
for index in np.arange(len(facility_temp)):
    if facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure > 300 psig':   
        metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure 100 to 300 psig':   
        metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure < 100 psig':   
        metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
# 2013 value and average to get 2014
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1002812]
index_13 = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2013].index[0]
index_14 = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2014].index[0]
metering_13 = facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS'].iloc[index_13]
metering_14 = round((metering_13+metering_15)/2)

# Replace 2014 below-grade metering value
facility_2014.at[index_14,'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS'] = metering_14

In [ ]:
# Problem 5:
# NY abrupt drop in stations from 2015 onwards. 
# Cause:
# Facility 1003642: below metering dropped from 823 to 48 2014-2015. Facility 1003576: below metering 
# dropped from 457 to 14 2014-2015. Corresponding huge drop in stations/mileage ratio. 
# EPA Solution:
# Use 2015 counts for 2012 (start_year)-2014.

# Replace above/below grade metering/transfer stations (4 categories total) with 2015 values

# 1) Facility 1003642

# Initialize counts for each category
above_metering_15 = 0
above_transfer_15 = 0
below_metering_15 = 0
below_transfer_15 = 0

# Count stations company reported in 2015
# Below-grade 2015
facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'] == 2015]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID']==1003642]
facility_temp.reset_index(drop=True,inplace=True)
for index in np.arange(len(facility_temp)):
    if facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig':   
        below_transfer_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure 100 to 300 psig':   
        below_transfer_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index] 
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure < 100 psig':   
        below_transfer_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure > 300 psig':   
        below_metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure 100 to 300 psig':   
        below_metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure < 100 psig':   
        below_metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
# Above-grade 2015
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR'] == 2015]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1003642]
facility_temp.reset_index(drop=True,inplace=True)
above_transfer_15 = facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'][0]
above_metering_15 = facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'][0]   

# Array of indices of 2012 (start year)-2014 entries for facility 1003642
index_array = np.zeros(2015-start_year)
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1003642]
for iyear in np.arange(2015-start_year):
    index_array[iyear] = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==start_year+iyear].index.values.astype(int)[0]

# Replace 2012-2014 with 2015 counts
for iyear in np.arange(2015-start_year):
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_METERING_STATIONS'] = above_metering_15
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_TRANSFER_STATIONS'] = above_transfer_15
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS'] = below_metering_15
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_TRANSFER_STATIONS'] = below_transfer_15

    
# 2) Facility 1003576

# Initialize counts for each category
above_metering_15 = 0
above_transfer_15 = 0
below_metering_15 = 0
below_transfer_15 = 0

# Count stations company reported in 2015
# Below-grade 2015
facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'] == 2015]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID']==1003576]
facility_temp.reset_index(drop=True,inplace=True)
for index in np.arange(len(facility_temp)):
    if facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig':   
        below_transfer_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure 100 to 300 psig':   
        below_transfer_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index] 
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure < 100 psig':   
        below_transfer_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure > 300 psig':   
        below_metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure 100 to 300 psig':   
        below_metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure < 100 psig':   
        below_metering_15 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
# Above-grade 2015
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR'] == 2015]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1003576]
facility_temp.reset_index(drop=True,inplace=True)
above_transfer_15 = facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'][0]
above_metering_15 = facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'][0]   

# Array of indices of 2012 (start year)-2014 entries for facility 1003642
index_array = np.zeros(2015-start_year)
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1003576]
for iyear in np.arange(2015-start_year):
    index_array[iyear] = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==start_year+iyear].index.values.astype(int)[0]
# Replace 2012-2014 with 2015 counts
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_METERING_STATIONS'] = above_metering_15
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_TRANSFER_STATIONS'] = above_transfer_15
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS'] = below_metering_15
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_TRANSFER_STATIONS'] = below_transfer_15


In [ ]:
# Problem 6:
# NC stations counts had huge drop in 2015. 
# Cause: 
# Company 1004567 decreased metering stations from 958 to 671, 1001583 decreased metering 
# stations from 1396 to 645. Corresponding drop in stations/mileage ratio.
# EPA Solution:
# Use 2016 for 2012 (start_year)-2016 counts

# Initialize counts for each category
above_metering_16 = 0
above_transfer_16 = 0
below_metering_16 = 0
below_transfer_16 = 0

# Count stations company reported in 2016
# Below-grade 2016
facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'] == 2016]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID']==1001583]
facility_temp.reset_index(drop=True,inplace=True)
for index in np.arange(len(facility_temp)):
    if facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig':   
        below_transfer_16 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure 100 to 300 psig':   
        below_transfer_16 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index] 
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure < 100 psig':   
        below_transfer_16 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure > 300 psig':   
        below_metering_16 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure 100 to 300 psig':   
        below_metering_16 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
    elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure < 100 psig':   
        below_metering_16 += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
# Above-grade 2016
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR'] == 2016]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1001583]
facility_temp.reset_index(drop=True,inplace=True)
above_transfer_16 = facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'][0]
above_metering_16 = facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'][0]  

# Array of indices of 2012 (start_year)-2014 entries for facility 1001583
index_array = np.zeros(2015-start_year)
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1001583]
for iyear in np.arange(2015-start_year):
    index_array[iyear] = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==start_year+iyear].index.values.astype(int)[0]
    # Replace 2012 (start_year)-2014 with 2016 counts
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_METERING_STATIONS'] = above_metering_16
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_TRANSFER_STATIONS'] = above_transfer_16
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS'] = below_metering_16
    facility_2014.at[int(index_array[iyear]),'W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_TRANSFER_STATIONS'] = below_transfer_16

    
# Replace 2015 with 2016 counts, 
# below-grade - zero out metering/transfer inputs and add 2016 total to one cell so it is counted later
for index in np.arange(len(facility_2018)):
    if facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'][index]==2015:
        if facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID'][index]==1001583:
            if facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig':   
                facility_2018.at[index,'EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'] = 0
            elif facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure 100 to 300 psig':   
                facility_2018.at[index,'EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'] = 0
            elif facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure < 100 psig':   
                facility_2018.at[index,'EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'] = 0
            elif facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure > 300 psig':   
                facility_2018.at[index,'EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'] = 0
            elif facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure 100 to 300 psig':   
                facility_2018.at[index,'EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'] = 0
            elif facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure < 100 psig':   
                facility_2018.at[index,'EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'] = 0
# Add in new below-grade
facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR']==2015]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.FACILITY_ID']==1001583]
index_below_15 = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE']=='Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig'].index.values.astype(int)[0]
facility_2018.at[int(index_below_15),'EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'] = below_metering_16 + below_transfer_16

# Change above-grade 2015 to 2016 values
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1001583]
index_15 = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR']==2015].index.values.astype(int)[0]

facility_above_2018.at[int(index_15),'EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'] = above_transfer_16
facility_above_2018.at[int(index_15),'EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'] = above_metering_16

In [ ]:
# Problem 7:
# UT huge increase in stations 2018, with corresponding increase in stations/mileage ratio; 
# Cause:
# facility 1004977 reported about 1000 more above-grade stations 2017-2018.
# EPA Solution:
# Use the 2017 above-grade value for 2018.

# Count stations company reported in 2017
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR'] == 2017]
facility_temp = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1004977]
facility_temp.reset_index(drop=True,inplace=True)
above_transfer_17 = facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'][0]
above_metering_17 = facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'][0]

# Replace 2018 values with 2017 values
facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.FACILITY_ID']==1004977]
index_18 = facility_temp[facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR']==2018].index.values.astype(int)[0]
facility_above_2018.at[int(index_15),'EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'] = above_transfer_17
facility_above_2018.at[int(index_15),'EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'] = above_metering_17

##### 2.5.3 Calculating State-Level GHGRP Metering/Transfer Station Counts

In [ ]:
# Sum the metering/transfer stations for each State

Stations_above = np.zeros((len(State_ANSI),num_years))
Stations_below = np.zeros((len(State_ANSI),num_years))

# for each reporting year, sum the station counts for each state

# a) Stations 2012 (start_year)-2014: above and below
for iyear in np.arange(2015-start_year):
    facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR'] == iyear+start_year]
    facility_temp.reset_index(drop=True,inplace=True)
    
    for index in np.arange(len(facility_temp)):
        match_state = np.where(State_ANSI['abbr'] == facility_temp['state'][index])[0][0]

        Stations_above[match_state][iyear] += facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_METERING_STATIONS'][index]
        Stations_above[match_state][iyear] += facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.ABOVE_GRADE_TRANSFER_STATIONS'][index]
        Stations_below[match_state][iyear] += facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_METERING_STATIONS'][index]
        Stations_below[match_state][iyear] += facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.BELOW_GRADE_TRANSFER_STATIONS'][index]

# b) Stations 2015-2018
# Below
for iyear in np.arange(2015-start_year,num_years):
    facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'] == iyear+start_year]
    facility_temp.reset_index(drop=True,inplace=True)
    
    for index in np.arange(len(facility_temp)):
        match_state = np.where(State_ANSI['abbr'] == facility_temp['state'][index])[0][0]
    
        if facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure > 300 psig':   
            Stations_below[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure 100 to 300 psig':   
            Stations_below[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index] 
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade T-D Station  Gas Service  Inlet Pressure < 100 psig':   
            Stations_below[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure > 300 psig':   
            Stations_below[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure 100 to 300 psig':   
            Stations_below[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Below Grade M-R Station  Gas Service  Inlet Pressure < 100 psig':   
            Stations_below[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]

# Above
for iyear in np.arange(2015-start_year,num_years):
    facility_temp = facility_above_2018[facility_above_2018['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.REPORTING_YEAR'] == iyear+start_year]
    facility_temp.reset_index(drop=True,inplace=True)
    
    for index in np.arange(len(facility_temp)):
        match_state = np.where(State_ANSI['abbr'] == facility_temp['state'][index])[0][0]

        Stations_above[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_TD_FACILITY_STATIONS'][index]
        Stations_above[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_NGDIST_LEAKS.TOTAL_NON_TD_FACILITY_STATIONS'][index]

##### 2.5.4 Fixing GHGRP Data Inconsistencies in Mileage Reporting
There are a few reporting inconsistencies in the GHGRP data.
Below are 2 known problems, their causes, EPA's proposed solutions, and their implementations.

In [ ]:
# Problem 1:
# GHGRP facility no. 1006503 didn't report mileage in 2012.
# EPA Solution:
# We have compared the GHGRP-reported data with the data the facility reported to the U.S. Department of Transportation
# – Pipeline and Hazardous Material Safety Administration (PHMSA). PHMSA-reported data appear similar to GHGRP reported
# data for 2013, but much higher (and more similar to 2013) than the GHGRP reported data for 2012. The 2012 and 2013 
# data in PHMSA appear to be similar so it seems like a good approach to extend back with the 2013-2018 GHGRP data.
# To do this, the the facility address has to be crosswalked with the PHMSA data. The operator ID for this particular 
# GHGRP reporter in the PHMSA dataset is 12408.

# Read 2012 PHMSA data
phmsa_2012 = pd.read_csv(phmsa_gas_distr_2012_inputfile,header=2)
phmsa_temp = phmsa_2012[phmsa_2012['OPERATOR_ID']==12408] #access row of company

# Store mileage values of unprotected steel, protected steel, plastic, and cast iron
usteel = phmsa_temp['MMILES_STEEL_UNP_BARE'].iloc[0]+phmsa_temp['MMILES_STEEL_UNP_COATED'].iloc[0]
psteel = phmsa_temp['MMILES_STEEL_CP_BARE'].iloc[0]+phmsa_temp['MMILES_STEEL_CP_COATED'].iloc[0]
plastic = phmsa_temp['MMILES_PLASTIC'].iloc[0]
castiron = phmsa_temp['MMILES_CI'].iloc[0]

# Find index of 2012 row for company 1006503
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1006503]
index_2012 = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2012].index.values.astype(int)[0]

# Change mileage values in facility_2014 to PHMSA values
facility_2014.at[index_2012,'W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_UNPR_STEEL_DIST_MAINS'] = usteel
facility_2014.at[index_2012,'W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PROT_STEEL_DIST_MAINS'] = psteel
facility_2014.at[index_2012,'W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PLSTIC_DIST_MAINS'] = plastic
facility_2014.at[index_2012,'W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_CAST_IRON_DIST_MAINS'] = castiron

In [ ]:
# Problem 2:
# Abrupt mileage jump 2012-2013 of ~3500 mi.
# Cause: 
# GHGRP facility 1003050 didn’t report plastic/protected steel mileage in 2012, but reports consistent nonzero 
# values for all later years.
# EPA Solution:
# Data from the facility are also available from PHMSA. PHMSA-reported data appear similar to GHGRP reported 
# data for 2013, but much higher (and more similar to 2013) the GHGRP reported data for 2012. The best solution 
# seems to be replacing the 2012 GHGRP data with data from PHMSA. The operator ID for this GHGRP reporter in 
# the PHMSA dataset is 31232.

# Read 2012 PHMSA data
phmsa_2012 = pd.read_csv(phmsa_gas_distr_2012_inputfile,header=2)
phmsa_temp = phmsa_2012[phmsa_2012['OPERATOR_ID']==31232] #access rows of company (there are 3 entries, last is most recent)
phmsa_temp.reset_index(inplace=True)
phmsa_temp.drop(labels=[0,1], axis=0, inplace=True) #isolate most recent entry

# Store mileage values of protected steel and plastic
psteel = phmsa_temp['MMILES_STEEL_CP_BARE'].iloc[0]+phmsa_temp['MMILES_STEEL_CP_COATED'].iloc[0]
plastic = phmsa_temp['MMILES_PLASTIC'].iloc[0]

# Find index of 2012 row for company 1003050
facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.FACILITY_ID']==1003050]
index_2012 = facility_temp[facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR']==2012].index.values.astype(int)[0]

# Change mileage values in facility_2014 to PHMSA values
facility_2014.at[index_2012,'W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PROT_STEEL_DIST_MAINS'] = psteel
facility_2014.at[index_2012,'W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PLSTIC_DIST_MAINS'] = plastic

##### 2.5.5 Sum GHGRP Pipeline Mileage Totals by State

In [ ]:
#Adding up mileage of pipelines from GHGRP by State
Mains_total = np.zeros((len(State_ANSI),num_years))

# For each year, add up the piepline mileage of each type in each state

# a) 2012 (start year)-2014: add up mains mileage total
for iyear in np.arange(2015-start_year):
    facility_temp = facility_2014[facility_2014['W_LOCAL_DIST_COMPANIES_DETAILS.REPORTING_YEAR'] == iyear+start_year]
    facility_temp.reset_index(drop=True,inplace=True)
    
    for index in np.arange(len(facility_temp)):
        match_state = np.where(State_ANSI['abbr'] == facility_temp['state'][index])[0][0]

        Mains_total[match_state][iyear] += facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_CAST_IRON_DIST_MAINS'][index]
        Mains_total[match_state][iyear] += facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PLSTIC_DIST_MAINS'][index]
        Mains_total[match_state][iyear] += facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_PROT_STEEL_DIST_MAINS'][index]
        Mains_total[match_state][iyear] += facility_temp['W_LOCAL_DIST_COMPANIES_DETAILS.MILES_OF_UNPR_STEEL_DIST_MAINS'][index]

# b) 2015-2018 (end year): add up mains mileage total
for iyear in np.arange(2015-start_year,num_years):
    facility_temp = facility_2018[facility_2018['EF_W_EQUIP_LEAKS_POP_COUNT.REPORTING_YEAR'] == iyear+start_year]
    facility_temp.reset_index(drop=True,inplace=True)
    
    for index in np.arange(len(facility_temp)):
        match_state = np.where(State_ANSI['abbr'] == facility_temp['state'][index])[0][0]
    
        if facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Distribution Mains  Gas Service - Cast Iron':   
            Mains_total[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Distribution Mains  Gas Service - Unprotected Steel':   
            Mains_total[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Distribution Mains  Gas Service - Protected Steel':   
            Mains_total[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]
        elif facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.EMISSION_SRC_TYPE'][index] == 'Distribution Mains  Gas Service - Plastic':   
            Mains_total[match_state][iyear] += facility_temp['EF_W_EQUIP_LEAKS_POP_COUNT.SOURCE_TYPE_COUNT'][index]


##### 2.5.6 Scaling GHGRP Station Count Data Using PHMSA

In [ ]:
# Some states have no reported station counts, even when reporting pipeline mileage. However, not all states
# report pipeline mileage either (CHECK ON THIS). In addition, GHGRP-reported 
# station counts for states with data may also be underestimates (due to threshold reporting requirements). 
# For States with GHGRP reported station counts, we scale the GHGRP station counts (above/below-grade) in each 
# state using the proportion of PHMSA pipeline mileage to GHGRP reported pipeline mileage (assuming that 
# any possible station under-reporting is proportional with any possible pipeline mileage underreporting.
# E.g., Total Station Counts = GHGRP Station counts * (PHMSA mileage/ GHGRP mileage). 
# For states with no GHGRP reported station counts, we take the average ratio of the GHGRP station counts 
# (above/below-grade) relative to the PHMSA mileage for states with GHGRP data, and then apply that ratio 
# to the other states without GHGRP station count data
# E.g., Station Counts = PHMSA mileage * (sum GHGRP station counts for all states with data/ sum PHMSA mileage for same states)

# Initialize state-level station count arrays
stations_above_state = np.zeros([len(State_ANSI),num_years])
stations_below_state = np.zeros([len(State_ANSI),num_years])
stations_state = np.zeros([len(State_ANSI),num_years])

# SET DIVIDE BY ZERO AND INVALID VALUE WARNING OFF
np.seterr(divide = 'ignore') 
np.seterr(invalid='ignore')

# a) States with GHGRP reported data: estimate total number of stations by scaling GHGRP reported mileage to PHMSA mileage 
for iyear in np.arange(num_years):
    for istate in np.arange(len(State_ANSI)):
        stations_above_state[istate,iyear] = Stations_above[istate,iyear] * \
                                            ( PHMSA_mains_total[istate,iyear] / Mains_total[istate,iyear])
        stations_below_state[istate,iyear] = Stations_below[istate,iyear] * \
                                            ( PHMSA_mains_total[istate,iyear] / Mains_total[istate,iyear])
    
#TURN ALL WARNINGS BACK ON
np.seterr('warn')

# will fill NaN if divide by zero error, replace with zero                                           
stations_above_state = np.nan_to_num(stations_above_state) 
stations_below_state = np.nan_to_num(stations_below_state) 

# b) States with no GHGRP reported data: estimate using the ratio of station counts to PHMSA milage for states with data
ratio_stations_above_pipeline = np.sum(stations_above_state[stations_above_state>0]) / np.sum(PHMSA_mains_total[stations_above_state>0])
ratio_stations_below_pipeline = np.sum(stations_below_state[stations_below_state>0]) / np.sum(PHMSA_mains_total[stations_below_state>0])
for iyear in np.arange(num_years):
    for istate in np.arange(len(State_ANSI)-6): #only consider states & DC, not territories
        if stations_above_state[istate,iyear]==0:
            stations_above_state[istate,iyear] = ratio_stations_above_pipeline * PHMSA_mains_total[istate,iyear]
        if stations_below_state[istate,iyear]==0:
            stations_below_state[istate,iyear] = ratio_stations_below_pipeline * PHMSA_mains_total[istate,iyear]
            
# Print time
ct = datetime.datetime.now() 
print("current time:", ct) 

-----------
## Step 3. Read in and Format US EPA GHGI Data
----------

In [ ]:
# Emissions are in units of MG (= 1x10-6 Tg, 1e-3 kt)

# METHANE
names = pd.read_excel(EPA_NG_inputfile, sheet_name = "Inventory Emissions", usecols = "A:AG", skiprows = 5, header = 0, nrows = 1)
colnames = names.columns.values
EPA_emi_dist_NG = pd.read_excel(EPA_NG_inputfile, sheet_name = "Inventory Emissions", usecols = "A:AG", skiprows = 205, names = colnames, nrows = 30)
EPA_emi_dist_NG= EPA_emi_dist_NG.drop(columns = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3'])
EPA_emi_dist_NG['Source']= EPA_emi_dist_NG['Source'].str.replace(r"\(","")
EPA_emi_dist_NG['Source']= EPA_emi_dist_NG['Source'].str.replace(r"\)","")
EPA_emi_dist_NG = EPA_emi_dist_NG.fillna('')
EPA_emi_dist_NG = EPA_emi_dist_NG.drop(columns = [*range(1990, start_year,1)])
EPA_emi_dist_NG.reset_index(inplace=True, drop=True)
display(EPA_emi_dist_NG)

#### Step 3.1.2. Read in Total Distribution Emissions

In [ ]:
# Read in total processing emissions (with methane reductions accounted for)
# data are in kt

#CH4
names = pd.read_excel(EPA_NG_inputfile, sheet_name = "SUMMARY CH4", usecols = "A:AD", skiprows = 10, header = 0, nrows = 1)
colnames = names.columns.values
EPA_emi_total_NG_CH4 = pd.read_excel(EPA_NG_inputfile, sheet_name = "SUMMARY CH4", usecols = "A:AD", skiprows = 17, names = colnames, nrows = 5)
EPA_emi_total_NG_CH4.rename(columns={EPA_emi_total_NG_CH4.columns[0]:'Source'}, inplace=True)
EPA_emi_total_NG_CH4 = EPA_emi_total_NG_CH4.drop(columns = [*range(1990, start_year,1)])
EPA_emi_total_NG_CH4.reset_index(inplace=True, drop=True)

print("EPA GHGI Emissions with Reductions (kt)")
display(EPA_emi_total_NG_CH4)

#### 3.1.3. Split Emissions into Gridding Groups (each Group will have the same proxy applied during the gridding)

In [ ]:
# Final Emissions in Units of kt
# Use mapping proxy and source files to split the GHGI emissions

DEBUG=1
start_year_idx = EPA_emi_dist_NG.columns.get_loc(start_year)
end_year_idx = EPA_emi_dist_NG.columns.get_loc(end_year)+1
sum_emi = np.zeros(num_years)

for igroup in np.arange(0,len(emi_group_names)): #loop through all groups, finding the GHGI sources in that group and summing emissions for that region, year
    vars()[emi_group_names[igroup]] = np.zeros([num_years])
    ##DEBUG ## print('here3',ghgi_dist_groups[igroup], np.shape(vars()[ghgi_dist_groups[igroup]]))
    source_temp = ghgi_dist_map.loc[ghgi_dist_map['GHGI_Emi_Group'] == emi_group_names[igroup], 'GHGI_Source']
    pattern_temp  = '|'.join(source_temp)
    ##DEBUG ## display(pattern_temp)
    emi_temp = EPA_emi_dist_NG[EPA_emi_dist_NG['Source'].str.contains(pattern_temp)]
    ##DEBUG ## display(emi_temp)
    vars()[emi_group_names[igroup]][:] = np.where(emi_temp.iloc[:,start_year_idx:] =='',[0],emi_temp.iloc[:,start_year_idx:]).sum(axis=0)/float(1000) #convert Mg to kt


#Check against total summary emissions 
print('QA/QC #1: Check Processing Emission Sum against GHGI Summary Emissions')
for iyear in np.arange(0,num_years): 
    for igroup in np.arange(0,len(emi_group_names)):
        sum_emi[iyear] += vars()[emi_group_names[igroup]][iyear]
            #print(np.sum(vars()[ghgi_prod_groups[igroup]][iyear]))
        
    summary_emi = EPA_emi_total_NG_CH4.iloc[4,iyear+1]  
    #Check 1 - make sure that the sums from all the regions equal the totals reported
    diff1 = abs(sum_emi[iyear] - summary_emi)/((sum_emi[iyear] + summary_emi)/2)
    if DEBUG==1:
        print(summary_emi)
        print(sum_emi[iyear])
    if diff1 < 0.0001:
        print('Year ', year_range[iyear],': PASS, difference < 0.01%')
    else:
        print('Year ', year_range[iyear],': FAIL (check Production & summary tabs): ', diff1,'%') 

----------------
## Step 4. Grid Emissions
---------------

### Step. 4.1. Calculate the monthly and regional weighted arrays

##### Step 4.1.1 Assign the Appropriate Proxy Variable Names

In [ ]:
# The names on the *left* need to match the 'NaturalGas_Distribution_ProxyMapping' 'Proxy_Group' names 
# (these are initialized in Step 2). 
# The names on the right are the variable names used to caluclate the proxies in this code.
# Names on the *right* need to match those from the code in Step 2.5

State_Main_CastIron = main_castiron
State_Main_Usteel = main_usteel
State_Main_Psteel = main_psteel
State_Main_Plastic = main_plastic
State_Serv_Usteel = serv_usteel
State_Serv_Psteel = serv_psteel
State_Serv_Plastic = serv_plastic
State_Serv_Copper = serv_copper
State_MR_Above = stations_above_state
State_MR_Below = stations_below_state
State_Residential = res_counts
State_CommIndu = indcom_counts
State_Meter = main_castiron + main_usteel + main_psteel + main_plastic

Map_population = pop_den_map*area_map

#### 4.1.2. Allocate National EPA Emissions to the State-Level
Allocation based on state-level pipeline mileage (PHMSA), leak volume (EIA), and station counts (PHMSA-corrected GHGRP data)

In [ ]:
# Calculate state-level emissions from all pieline types, service and metering stations, and leak losses.
# Emissions are calculated as a running sum of the as the national emissions, weighted a) by the pipeline
# mileage in each state relative to the national total (by type and for all types), by b) the number of service/metering 
# stations in each state relative to the national total, and by c) the leak volume in each state relative
# to the national total. Emissions in kt

DEBUG =1

# Data here are from the:
# 1) most recent EPA GHGI (emi_main_[type], emi_serv_[type], emi_mr_[above/below], emi_leak, emi_meter, )
# 2) yearly EIA leak loss volume data (leakvolume)
# 3) pre-2015 and more recent GHGRP data, scaled by PHMSA data (stations_[below/above]_state)
# 4) most recent PHMSA pipeline milaege data (main_[type], serv_[type])
# also note that national emissions are retained for groups that do not have state proxies identified in the mapping file
# and are later gridded in the next step
for igroup in np.arange(0,len(proxy_dist_map)):
    vars()['State_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']] = np.zeros([len(State_ANSI),num_years])
    vars()['NonState_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']] = np.zeros([num_years])
        
#Loop over years
for iyear in np.arange(num_years):
    #Loop over states
    for istate in np.arange(len(State_ANSI)):
        for igroup in np.arange(0,len(proxy_dist_map)):    
            if proxy_dist_map.loc[igroup,'State_Proxy_Group'] != '-':
                vars()['State_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][istate,iyear] = vars()[proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][iyear] * \
                    data_fn.safe_div(vars()[proxy_dist_map.loc[igroup,'State_Proxy_Group']][istate,iyear], np.sum(vars()[proxy_dist_map.loc[igroup,'State_Proxy_Group']][:,iyear]))
            else:
                vars()['NonState_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][iyear] = vars()[proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][iyear]
                
# Check sum of all gridded emissions + emissions not included in gridding (e.g., AK), and other non-gridded areas
print('QA/QC #1: Check weighted emissions against GHGI')   
for iyear in np.arange(0,num_years):
    summary_emi = EPA_emi_total_NG_CH4.iloc[4,iyear+1]
    calc_emi = 0
    for igroup in np.arange(0,len(proxy_dist_map)):
        calc_emi +=  np.sum(vars()['State_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][:,iyear])+\
            vars()['NonState_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][iyear] #np.sum(Emissions[:,iyear]) + Emissions_nongrid[iyear] + Emissions_nonstate[iyear]
    if DEBUG==1:
        print(summary_emi)
        print(calc_emi)
    diff = abs(summary_emi-calc_emi)/((summary_emi+calc_emi)/2)
    if diff < 0.0002:
        print('Year ', year_range[iyear], ': PASS, difference < 0.01%')
    else:
        print('Year ', year_range[iyear], ': FAIL -- Difference = ', diff*100,'%')

#### Step 4.1.3. Calculate Gridded Emissions (0.1x0.1 degrees)

In [ ]:
# Allocate State-Level emissions (kt) onto a 0.01x0.01 grid (weighted by population in each grid cell relative to state total)

#Define emission arrays
Emissions_array = np.zeros([area_map.shape[0],area_map.shape[1],num_years])
Emissions_array_01 = np.zeros([len(Lat_01),len(Lon_01),num_years])
Emissions_nongrid = np.zeros(num_years)

DEBUG = 1

# For each year, (2a) distribute state-level emissions onto a grid using proxies defined above ....
# To speed up the code, masks are used rather than looping individually through each lat/lon. 
# In this case, a mask of 1's is made for the grid cells that match the ANSI values for a given state
# The masked values are set to zero, remaining values = 1. 
# AK and HI and territories are removed from the analysis at this stage. 
# The emissions allocated to each state are at 0.01x0.01 degree resolution, as required to calculate accurate 'mask'
# arrays for each state. 
# (2b - NA here) For emission groups that were not first allocated to states, national emissions for those groups are gridded
# based on the relevant gridded proxy arrays (0.1x0.1 resolution). These emissions are at 0.1x0.1 degrees resolution. 
# (2c - NA here) - record 'not mapped' emission groups in the 'non-grid' array

print('**QA/QC Check: Sum of national gridded emissions vs. GHGI national emissions')
for igroup in np.arange(len(proxy_dist_map)):
    vars()['Ext_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']] = np.zeros([len(Lat_01),len(Lon_01),num_years])
    
for iyear in np.arange(0,num_years):
    if year_range[iyear]==2012 or year_range[iyear]==2016:
        year_days = np.sum(month_day_leap)
    else:
        year_days = np.sum(month_day_nonleap)

    #Step 1 - grid state level emissions with appropriate proxies 
    #running_count = 0
    for istate in np.arange(0,len(State_ANSI)):
        mask_state = np.ma.ones(np.shape(state_ANSI_map))
        mask_state = np.ma.masked_where(state_ANSI_map != State_ANSI['ansi'][istate], mask_state)
        mask_state = np.ma.filled(mask_state,0)   
        #print("state " + str(istate) +' of '+ str(len(State_ANSI)))
        for igroup in np.arange(len(proxy_dist_map)):
            proxy_temp = vars()[proxy_dist_map.loc[igroup,'Proxy_Group']]
            
            # 2a
            if proxy_dist_map.loc[igroup,'State_Proxy_Group'] != '-':
                if np.sum(mask_state*proxy_temp) > 0 and State_ANSI['abbr'][istate] not in {'AK','HI'} and istate < 51: 
                    weighted_array = data_fn.safe_div(mask_state*proxy_temp, np.sum(mask_state*proxy_temp))
                    emi_temp = vars()['State_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][istate,iyear]*weighted_array
                    Emissions_array[:,:,iyear] += emi_temp
                    vars()['Ext_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][:,:,iyear] += data_fn.regrid001_to_01(emi_temp[:,:], Lat_01, Lon_01)
                else:
                    Emissions_nongrid[iyear] += vars()['State_'+proxy_dist_map.loc[igroup,'GHGI_Emi_Group']][istate][iyear] 
    
    Emissions_array_01[:,:,iyear] += data_fn.regrid001_to_01(Emissions_array[:,:,iyear], Lat_01, Lon_01) #covert to 10x10km
    calc_emi = np.sum(Emissions_array_01[:,:,iyear]) + Emissions_nongrid[iyear]
    summary_emi = EPA_emi_total_NG_CH4.iloc[4,iyear+1]
    emi_diff = abs(summary_emi-calc_emi)/((summary_emi+calc_emi)/2)
    if DEBUG ==1:
        print(calc_emi)
        print(summary_emi)
    if abs(emi_diff) < 0.0001:
        print('Year '+ year_range_str[iyear]+': Difference < 0.01%: PASS')
    else: 
        print('Year '+ year_range_str[iyear]+': Difference > 0.01%: FAIL, diff: '+str(emi_diff))


#### Step 4.1.4 Save gridded emissions (kt)

In [ ]:
#save gridded emissions for each gridding group - for extension

#Initialize file
data_IO_fn.initialize_netCDF(grid_emi_outputfile, netCDF_description, 0, year_range, loc_dimensions, Lat_01, Lon_01)

unique_groups = np.unique(proxy_dist_map['GHGI_Emi_Group'])
unique_groups = unique_groups[unique_groups != 'Emi_not_mapped']

nc_out = Dataset(grid_emi_outputfile, 'r+', format='NETCDF4')

for igroup in np.arange(0,len(unique_groups)):
    print('Ext_'+unique_groups[igroup])
    if len(np.shape(vars()['Ext_'+unique_groups[igroup]])) ==4:
        ghgi_temp = np.sum(vars()[unique_groups[igroup]],axis=3) #sum month data if data is monthly
    else:
        ghgi_temp = vars()['Ext_'+unique_groups[igroup]]

    # Write data to netCDF
    data_out = nc_out.createVariable('Ext_'+unique_groups[igroup], 'f8', ('lat', 'lon','year'), zlib=True)
    data_out[:,:,:] = ghgi_temp[:,:,:]

#save nongrid data to calculate non-grid fraction extension
data_out = nc_out.createVariable('Emissions_nongrid', 'f8', ('year'), zlib=True)  
data_out[:] = Emissions_nongrid[:]
nc_out.close()

#Confirm file location
print('** SUCCESS **')
print("Gridded emissions (kt) written to file: {}" .format(os.getcwd())+grid_emi_outputfile)
print(' ')

del data_out, ghgi_temp, nc_out

#### 4.2 Calculate Gridded Emission Fluxes (molec./cm2/s) (0.1x0.1)

In [ ]:
#Convert emissions to emission flux
# convert kt to molec/cm2/s

DEBUG = 1

Flux_array_01 = np.zeros([len(Lat_01), len(Lon_01),num_years])
print('**QA/QC Check: Sum of national gridded emissions vs. GHGI national emissions')
  
for iyear in np.arange(0,num_years):
    if year_range[iyear]==2012 or year_range[iyear]==2016:
        year_days = np.sum(month_day_leap)
    else:
        year_days = np.sum(month_day_nonleap)

    conversion_factor_01 = 10**9 * Avogadro / float(Molarch4 *year_days * 24 * 60 *60) / area_matrix_01
    Flux_array_01[:,:,iyear] = Emissions_array_01[:,:,iyear]*conversion_factor_01
    #convert back to mass to check
    
    calc_emi =  np.sum(Flux_array_01[:,:,iyear]/conversion_factor_01)+Emissions_nongrid[iyear] #+np.sum(Flux_array[:,:,iyear]/conversion_factor) 
    summary_emi = EPA_emi_total_NG_CH4.iloc[4,iyear+1]
    emi_diff = abs(summary_emi-calc_emi)/((summary_emi+calc_emi)/2)
    if DEBUG ==1:
        print(calc_emi)
        print(summary_emi)
    if abs(emi_diff) < 0.0001:
        print('Year '+ year_range_str[iyear]+': Difference < 0.01%: PASS')
    else: 
        print('Year '+ year_range_str[iyear]+': Difference > 0.01%: FAIL, diff: '+str(emi_diff))
        
Flux_Emissions_Total_annual = Flux_array_01

--------------
## Step 5. Write Gridded Data to File
------------------

In [ ]:
#initialize netCDF file
data_IO_fn.initialize_netCDF(gridded_outputfile, netCDF_description, 0, year_range, loc_dimensions, Lat_01, Lon_01)

# Write data to netCDF
nc_out = Dataset(gridded_outputfile, 'r+', format='NETCDF4')
nc_out.variables['emi_ch4'][:,:,:] = Flux_Emissions_Total_annual
nc_out.close()
#Confirm file location
print('** SUCCESS **')
print("Gridded gas distribution fluxes written to file: {}" .format(os.getcwd())+gridded_outputfile)


--------------
## Step 6. Plot Gridded Data
----------------

#### 6.1 Plot Annual Emission Fluxes

In [ ]:
scale_max = 10
save_flag = 0
save_outfile = ''
data_plot_fn.plot_annual_emission_flux_map(Flux_Emissions_Total_annual, Lat_01, Lon_01, year_range, title_str, scale_max,save_flag,save_outfile)

#### 6.2 Plot Difference Between First and Last Inventory Year

In [ ]:
# Plot difference between last and first year
save_flag = 0
save_outfile = ''
data_plot_fn.plot_diff_emission_flux_map(Flux_Emissions_Total_annual, Lat_01, Lon_01, year_range, title_diff_str,save_flag,save_outfile)

#### 6.3 Plot Activity Data Heat Maps

In [ ]:
print(np.shape(Map_population))
Map_population_01 = data_fn.regrid001_to_01(Map_population, Lat_01, Lon_01)
Map_temp = np.zeros([len(Lat_01),len(Lon_01),num_years])
for iyear in np.arange(0,num_years):
    Map_temp[:,:,iyear] = Map_population_01
print(np.shape(Map_temp))

In [ ]:
#Map (population) distribution mile heatmap (distribution mileage is at the state level, how to map?)

# Activity_Map = 0.1x0.1 map of activity data (counts or absolute units)
# Plot_Frac    = 0 or 1 (0= plot activity data in absolute counts, 1= plot fractional activity data)
# Lat          = 0.1 degree Lat values (select range)
# Lon          = 0.1 degree Lon values (select range)
# year_range   = array of inventory years
# title_str    = title of map
# legend_str   = title of legend
# scale_max    = maximum of color scale

Activity_Map = Map_temp
Plot_Frac = 1
Lat = Lat_01
Lon = Lon_01
year_range = year_range
title_str2 = "Proxy - Population"
legend_str = "Annual Fraction of National Population"
scale_max = 0.001

for iyear in np.arange(0,1):#len(year_range)): 
    my_cmap = copy(plt.cm.get_cmap('rainbow',lut=3000))
    my_cmap._init()
    slopen = 200
    alphas_slope = np.abs(np.linspace(0, 1.0, slopen))
    alphas_stable = np.ones(3003-slopen)
    alphas = np.concatenate((alphas_slope, alphas_stable))
    my_cmap._lut[:,-1] = alphas
    my_cmap.set_under('gray', alpha=0)
    
    Lon_cor = Lon[50:632]-0.05
    Lat_cor = Lat[43:300]-0.05
    
    xpoints = Lon_cor
    ypoints = Lat_cor
    yp,xp = np.meshgrid(ypoints,xpoints)
    
    if np.shape(Activity_Map)[0] == len(year_range):
        if Plot_Frac ==1:
            zp = Activity_Map[iyear,43:300,50:632]/np.sum(Activity_Map[iyear,:,:])
        else:
            zp = Activity_Map[iyear,43:300,50:632]
    elif np.shape(Activity_Map)[2] == len(year_range):
        if Plot_Frac ==1:
            zp = Activity_Map[43:300,50:632,iyear]/np.sum(Activity_Map[:,:,iyear])
        else: 
            zp = Activity_Map[43:300,50:632,iyear]
    #zp = zp/float(10**6 * Avogadro) * (year_days * 24 * 60 * 60) * Molarch4 * float(1e10)
    
    fig, ax = plt.subplots(dpi=300)
    m = Basemap(llcrnrlon=xp.min(), llcrnrlat=yp.min(), urcrnrlon=xp.max(),
                urcrnrlat=yp.max(), projection='merc', resolution='h', area_thresh=5000)
    m.drawmapboundary(fill_color='Azure')
    m.fillcontinents(color='FloralWhite', lake_color='Azure',zorder=1)
    m.drawcoastlines(linewidth=0.5,zorder=3)
    m.drawstates(linewidth=0.25,zorder=3)
    m.drawcountries(linewidth=0.5,zorder=3)
        
        #if Plot_Frac == 1:
        #    scale_max 
    
    xpi,ypi = m(xp,yp)
    plot = m.pcolor(xpi,ypi,zp.transpose(), cmap=my_cmap, vmin=10**-15, vmax=scale_max, snap=True,zorder=2)
    #plot = m.scatter(xpi,ypi,s=20,c=zp.transpose(),cmap=my_cmap,zorder=2,vmin = 10**-15,snap = True,vmax = scale_max)
    cb = m.colorbar(plot, location = "bottom", pad = "1%")        
    tick_locator = ticker.MaxNLocator(nbins=5)
    cb.locator = tick_locator
    cb.update_ticks()
    
    cb.ax.set_xlabel(legend_str,fontsize=10)
    cb.ax.tick_params(labelsize=10)
    Titlestring = str(year_range[iyear])+' '+title_str2
    plt.title(Titlestring, fontsize=14);
    plt.show();

In [ ]:
ct = datetime.datetime.now() 
ft = ct.timestamp() 
time_elapsed = (ft-it)/(60*60)
print('Time to run: '+str(time_elapsed)+' hours')
print('** GEPA_1B2b_Distribution: COMPLETE **')